# Start RSI model
https://chatgpt.com/c/67f7d833-ddf4-800a-87be-ea47a130482b

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
# from tensorflow.keras.models import Sequential
from keras.src.models import Sequential
# from tensorflow.keras.layers import LSTM, Dense
from keras.src.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

In [ ]:
from ta.momentum import RSIIndicator